# Task 1: Data Wrangling

### Table of Contents

1. [Introduction](#introduction)
2. [Data Gathering](#data_gathering)
3. [Data Assessing](#data_assessing)
    * [Assessment Summary](#assessment_summary)
4. [Data Cleaning](#data_cleaning)
5. [Saving](#saving)

## Introduction <a class="anchor" id="introduction"></a>

Data wrangling is performed on the Ames Housing dataset, which describes the features of residential homes in Ames, Iowa. The dataset is separated in two files, namely training set and testing set. Both training and testing sets have 79 columns describing the features of the homes, but training set has one additional column, which consists of the home prices. The explanation for each column can be found [here](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data).

The steps for data wrangling includes gathering, assessing, cleaning. Data wrangling is only performed on the training set only because testing set should be considered as unseen data. For gathering, data is only gathered from one source, which is the training set. Subsequently, data is assessed with visual and programmatic assessments to look for data quality and tidiness issues. Finally, data is cleaned based on the issues detected during assessment stage.

## Data Gathering <a class="anchor" id="data_gathering"></a>

In [1]:
# Import libraries
import os
import pandas as pd
import numpy as np
import math
pd.set_option('display.max_columns', None)  

In [2]:
# Define URLs of training sets
dirname = '/kaggle/input'
subdirname = 'house-prices-advanced-regression-techniques'
train_filename = 'train.csv'
train_filepath = os.path.join(dirname, subdirname, train_filename)

# Load training and testing sets
df = pd.read_csv(train_filepath)

In [3]:
# Print out the first 5 rows of df_train
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
print("Training set: {}".format(df.shape))

Training set: (1460, 81)


## Data Assessing <a class="anchor" id="data_accessing"></a>

Data is assessed with visual and programmatic assessment to look for data quality and tidiness issues

### Visual Assessment

In [5]:
# Sample n rows at random from the data frame for visual assessment
n_samples = 10
df.sample(n = n_samples)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
659,660,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,Edwards,Norm,Norm,1Fam,1Story,5,7,1964,1999,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,Gd,PConc,TA,TA,No,BLQ,637,Unf,0,849,1486,GasA,Ex,Y,SBrkr,1486,0,0,1486,1,0,1,0,3,1,TA,7,Typ,0,NaN,Detchd,1968.0,Fin,2,480,TA,TA,Y,0,0,0,0,0,0,NaN,MnPrv,NaN,0,3,2009,WD,Normal,167000
726,727,20,RL,NaN,21695,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,1Story,6,9,1988,2007,Hip,CompShg,Wd Sdng,Plywood,BrkFace,260.0,Gd,Gd,CBlock,Gd,TA,No,GLQ,808,Unf,0,72,880,GasA,Ex,Y,SBrkr,1680,0,0,1680,1,0,2,0,3,1,Gd,5,Typ,1,Gd,Attchd,1988.0,Fin,2,540,TA,TA,Y,292,44,0,182,0,0,NaN,NaN,NaN,0,12,2009,WD,Normal,222000
720,721,120,RL,NaN,6563,Pave,NaN,IR1,Low,AllPub,CulDSac,Mod,StoneBr,Norm,Norm,1Fam,1Story,8,5,1985,1985,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,Gd,GLQ,1148,Unf,0,594,1742,GasA,TA,Y,SBrkr,1742,0,0,1742,1,0,2,0,2,1,Gd,5,Typ,1,TA,Attchd,1985.0,RFn,2,564,TA,TA,Y,114,28,234,0,0,0,NaN,NaN,NaN,0,12,2006,WD,Normal,275000
648,649,60,RL,70.0,7700,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,PosN,Norm,1Fam,2Story,6,5,1966,1966,Gable,CompShg,MetalSd,MetalSd,BrkFace,351.0,TA,TA,CBlock,TA,TA,No,Unf,0,Unf,0,756,756,GasA,TA,Y,SBrkr,1051,788,0,1839,0,0,1,1,4,1,TA,7,Typ,1,TA,Attchd,1966.0,Unf,2,442,TA,TA,Y,0,124,216,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal,155000
1365,1366,60,FV,NaN,7500,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,2Story,7,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,533,Unf,0,281,814,GasA,Ex,Y,SBrkr,814,860,0,1674,1,0,2,1,3,1,Gd,7,Typ,0,NaN,Attchd,2000.0,RFn,2,663,TA,TA,Y,0,96,0,0,0,0,NaN,NaN,NaN,0,1,2010,WD,Normal,216000
121,122,50,RM,50.0,6060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1.5Fin,4,5,1939,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,TA,TA,PConc,TA,TA,No,Unf,0,Unf,0,732,732,GasA,Gd,Y,SBrkr,772,351,0,1123,0,0,1,0,3,1,TA,4,Typ,0,NaN,Detchd,1979.0,Unf,1,264,TA,TA,P,0,0,140,0,0,0,NaN,MnPrv,NaN,0,6,2007,WD,Normal,100000
499,500,20,RL,70.0,7535,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1958,1985,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,No,BLQ,111,LwQ,279,522,912,GasA,Fa,Y,SBrkr,912,0,0,912,0,1,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1958.0,Fin,1,297,TA,TA,Y,12,285,0,0,0,0,NaN,MnWw,Shed,480,6,2007,WD,Normal,120000
1066,1067,60,RL,59.0,7837,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,7,1993,1994,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,799,799,GasA,Gd,Y,SBrkr,799,772,0,1571,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1993.0,RFn,2,380,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,5,2009,WD,Normal,178000
767,768,50,RL,75.0,12508,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,6,7,1940,1985,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,660,Unf,0,323,983,GasA,Ex,Y,SBrkr,983,767,0,1750,1,0,2,0,4,1,TA,7,Mod,0,NaN,Attchd,1989.0,Unf,1,423,TA,TA,Y,245,0,156,0,0,0,NaN,NaN,Shed,1300,7,2008,WD,Normal,160000
215,216,20,RL,72.0,10011,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1957,1996,Gable,CompShg,HdBoard,HdBoard,BrkFace,64.0,TA,TA,CBlock,TA,TA,No,BLQ,360,Unf,0,

### Programmatic Assessment

#### Step 1: Check if there are any duplicates

In [6]:
# Check for duplicates based on all columns
print("Number of duplicates: {}".format(sum(df.duplicated())))

# Check for duplicates based on all columns except 'Id'
columns_without_id = list(df.columns)
columns_without_id.remove('Id')

print("Number of duplicates (without 'Id'): {}".format(sum(df.duplicated(subset=columns_without_id))))

Number of duplicates: 0
Number of duplicates (without 'Id'): 0


> ##### Finding(s) for Step 1:
> - There is no duplicate.


#### Step 2: Determine the number of nulls/missing data and the correct datatypes

In [7]:
# Check number of NaNs in each column and data type of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

> ##### Finding(s) for Step 2:

> - 80 variables (excld. ID)

>> -  34 Numeric variables 

>>> -  14 discrete variables (YearBuilt, YearRemodAdd, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, TotRmsAbvGrd, Fireplaces, GarageYrBlt, GarageCars, MoSold, YrSold)

>>> -  20 continuous variables (LotFrontage, LotArea, MasVnrArea, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, GarageArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal, SalePrice)

>> - 46 categorical variables 

>>> - 25 nominal variables (MSSubClass, MSZoning, Street, Alley, Utilities, LotConfig, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, Foundation, Heating, CentralAir, Electrical, GarageType, PavedDrive, MiscFeature, SaleType, SaleCondition)

>>> - 21 ordinal variables (LotShape, LandContour, LandSlope, OverallQual, OverallCond, ExterQual, ExterCond, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, HeatingQC, KitchenQual, Functional, FireplaceQu, GarageFinish, GarageQual, GarageCond, PoolQC, Fence)

> - Wrong data types (MSSubClass) - should be string instead of integer

> - Wrong data types (LotArea, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal) - should be float instead of integer

> - Wrong data types (BsmtFullBath, BsmtHalfBath, GarageYrBlt) - should be integer instead of float

> - 19 variables with nulls/missing data

>> - 3 numeric variables (LotFrontage, MasVnrArea,  GarageYrBlt)

>> - 16 categorical variables (Alley, MasVnrType, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1,BsmtFinType2, Electrical, FireplaceQu, GarageType, GarageFinish, GarageQual, GarageCond, PoolQC, Fence, MiscFeature)

#### Step 3: Investigate the missing data further

Based on data_description_txt (can be found [here](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)), certain variables consist of nulls or NANs because these variables are not possessed by the house. For example, NAN for alley variable means that the house does not have alley access. Thus, null or NAN doesn't mean missing data for these variables. 

> ##### Finding(s) for Step 3: 

> - Nulls in 23 variables do not represent missing data 

>> - 9 numeric variable (BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath, BsmtHalfBath, GarageYrBlt, GarageCars, GarageArea)

>> - 14 categorical variable (Alley, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, FireplaceQu, GarageType, GarageFinish, GarageQual, GarageCond, PoolQC, Fence, MiscFeature)

#### Step 4: Look for any outliers in numeric variables

In [8]:
# Compute statistics of numeric variables
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


> ##### Finding(s) for Step 4:

> - The data for numeric variables are pretty clean as there isn't any obvious outlier.

#### Step 5: Look for any abnormal data in categorical variables

In [9]:
# Define categorical variables
cat_vars = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 
            'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 
            'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical', 'GarageType', 'PavedDrive', 
            'MiscFeature', 'SaleType', 'SaleCondition', 'LotShape', 'LandContour', 'LandSlope', 'OverallQual', 
            'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
            'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual',
            'GarageCond', 'PoolQC', 'Fence']

# Print the unique values for each categorical variable
for var in cat_vars:
    # Get unique values
    unique_vals = df[var].unique()
 
    print("{} : {}".format(var, unique_vals))

MSSubClass : [ 60  20  70  50 190  45  90 120  30  85  80 160  75 180  40]
MSZoning : ['RL' 'RM' 'C (all)' 'FV' 'RH']
Street : ['Pave' 'Grvl']
Alley : [nan 'Grvl' 'Pave']
Utilities : ['AllPub' 'NoSeWa']
LotConfig : ['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
Neighborhood : ['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
Condition1 : ['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
Condition2 : ['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
BldgType : ['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
HouseStyle : ['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']
RoofStyle : ['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']
RoofMatl : ['CompShg' 'WdShngl' 'Metal' 'WdShake' 'Membran' 'Tar&Grv' 'Roll'
 'ClyTile']
Exterior1st : ['VinylSd' 'MetalSd' '

> ##### Finding(s) for Step 5:

> - After comparing the list of unique values of each variable with data_description_txt (can be found [here](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)), the data for categorical variables are pretty clean and only the following two data quality issues were found.

>> - 'WdShing' in 'Exterior1st' variable is the same as 'Wd Shng' in 'Exterior2nd' variable. For consistency, 'Wd Shng' in 'Exterior2nd' should be renamed as 'WdShing'

>> - 'BrkComm' in 'Exterior1st' variable is the same as 'Brk Cmn' in 'Exterior2nd' variable. For consistency, 'Brk Cmn' in 'Exterior2nd' should be renamed as 'BrkComm'

#### Step 6: Look for any discrepancies in year variables

There are 4 variables that are related to year, namely YearBuilt, YearRemodAdd, GarageYrBlt and YrSold. 

- YearBuilt:  Original construction date
- YearRemodAdd: Remodel date
- GarageYrBlt: Year garage was built
- YrSold: Year Sold

These 4 variables will be compared to ensure the chronological order of these variables is logical. YearBuilt should occur first, and YrSold should occur last. YearRemodAdd and GarageYrBlt should occur within YearBuilt and YrSold. Thus, the expected chronological order is YearBuilt -> YearRemodAdd/GarageYrBuilt -> YrSold


In [10]:
# Define year variables
year_vars = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']

# Select YearBuilt, YearRemodAdd, GarageYrBlt and YrSold from the data frame
df_year = df[year_vars]

# Define a function to check the chronological order
def is_correct_order(x):
    # Set 'YearRemodAdd' or 'YearGarageYrBlt' as 'YearBuilt if they are NaN because NaN will cause
    # the condition for checking chronological order below to return False in any scenario.
    if pd.isna(x['YearRemodAdd']):
        x['YearRemodAdd'] = x['YearBuilt'] 
        
    if pd.isna(x['GarageYrBlt']):
        x['GarageYrBlt'] = x['YearBuilt'] 
    
    # Check correctness of chronological order
    if x['YearBuilt'] <= x['YearRemodAdd'] <= x['YrSold'] and x['YearBuilt'] <= x['GarageYrBlt'] <= x['YrSold']:
        return True
    else:
        return False

# Get row(s) whose chronological order is not logical
df_year[~(df_year.apply(is_correct_order, axis=1))]

,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold
29,1927,1950,1920.0,2008
93,1910,1998,1900.0,2007
324,1967,2007,1961.0,2010
523,2007,2008,2007.0,2007
600,2005,2005,2003.0,2006
736,1950,1950,1949.0,2006
1103,1959,1959,1954.0,2006
1376,1930,1950,1925.0,2008
1414,1923,2000,1922.0,2008
1418,1963,1963,1962.0,2008


> ##### Finding(s) for Step 6:

> - For rows with ID of 523, YearRemodAdd doesn't occur between YearBuilt and YrSold. YearRemodAdd should be replaced with YearBuilt or YrSold, depending on which is closer to YearRemodAdd

> - For rows with IDs of 29, 93, 324, 600, 736, 1103, 1376, 1414 and 1418, GarageYrBlt doesn't occur between YearBuilt and YrSold. GarageYrBlt should be replaced with YearBuilt or YrSold, depending on which is closer to GarageYrBlt

#### Step 7: Check consistency in area (square feet)

If the data is correct, the following equations must be satisfied.

- TotalBsmtSF = BsmtFinSF1 + BsmtFinSF2 + BsmtUnfSF
- GrLivArea = 1stFlrSF + 2ndFlrSF + LowQualFinSF

In [11]:
# Get the row(s) whose TotalBsmtSF = BsmtFinSF1 + BsmtFinSF2 + BsmtUnfSF is not satisfied
df[df['BsmtFinSF1'] + df['BsmtFinSF2'] + df['BsmtUnfSF'] != df['TotalBsmtSF']]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


In [12]:
# Get the row(s) whose GrLivArea = 1stFlrSF + 2ndFlrSF + LowQualFinSF is not satisfied
df[df['1stFlrSF'] + df['2ndFlrSF'] + df['LowQualFinSF'] != df['GrLivArea']]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


> ##### Finding(s) for Step 7:

> - Based on the results shown above, all the rows satisfy both 'TotalBsmtSF = BsmtFinSF1 + BsmtFinSF2 + BsmtUnfSF' and 'GrLivArea = 1stFlrSF + 2ndFlrSF + LowQualFinSF'

#### Step 8: Look for variables with many nulls

In [13]:
# Set the threshold for acceptable null percentage
threshold = 0.2

# Print the variables that have null percentage greater than the threshold
df.loc[:, df.isnull().mean() > threshold].isnull().mean()

Alley          0.937671
FireplaceQu    0.472603
PoolQC         0.995205
Fence          0.807534
MiscFeature    0.963014
dtype: float64

> ##### Finding(s) for Step 8:

> - There are 5 variables with null percentage that is greater than 0.2 or 20% - Alley, FireplaceQu, PoolQC, Fence and MiscFeature. As these variables have too many missing data, they will be dropped from the data frame.

### Assessment Summary <a class="anchor" id="assessment_summary"></a>

Based on the findings above, the following data quality issues were detected.

- Wrong data types (MSSubClass) - should be string instead of integer
- Wrong data types (LotArea, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal) - should be float instead of integer
- Wrong data types (BsmtFullBath, BsmtHalfBath, GarageYrBlt) - should be integer instead of float
- Nulls in 23 variables do not represent missing data, but they means that these variables are not possessed by the house. Thus, nulls (if there are any) in these variables should be replaced with other string to indicate that these variables are not possessed by the house.
> - 9 numeric variable (BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath, BsmtHalfBath, GarageYrBlt, GarageCars, GarageArea)
> - 14 categorical variable (Alley, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, FireplaceQu, GarageType, GarageFinish, GarageQual, GarageCond, PoolQC, Fence, MiscFeature)

- 'WdShing' in 'Exterior1st' variable is the same as 'Wd Shng' in 'Exterior2nd' variable. For consistency, 'Wd Shng' in 'Exterior2nd' should be renamed as 'WdShing'
- 'BrkComm' in 'Exterior1st' variable is the same as 'Brk Cmn' in 'Exterior2nd' variable. For consistency, 'Brk Cmn' in 'Exterior2nd' should be renamed as 'BrkComm'
- For rows with ID of 523, YearRemodAdd doesn't occur between YearBuilt and YrSold. YearRemodAdd should be replaced with YearBuilt or YrSold, depending on which is closer to YearRemodAdd
- For rows with IDs of 29, 93, 324, 600, 736, 1103, 1376, 1414 and 1418, GarageYrBlt doesn't occur between YearBuilt and YrSold. GarageYrBlt should be replaced with YearBuilt or YrSold, depending on which is closer to GarageYrBlt
- Alley, FireplaceQu, PoolQC, Fence and MiscFeature should be dropped from the data frame as they have too many missing data.

## Data Cleaning <a class="anchor" id="data_cleaning"></a>

Data is cleaned based on the assessment summary. There are three steps for each cleaning prcoess, which are define, code, test.

- Define: define objective of the cleaning process
- Code: write code for performing the objective
- Test: verify cleaning process is carried out as intended

Before cleaning begins, a copy of the data frame is created. All the cleaning is performed on the copy, so that cleaned and uncleaned data frames can be compared if needed.

In [14]:
# Create a copy of the data frame
df_clean = df.copy()

#### Step 1: Change data types

***Define***

- Convert data type of MSSubClass from integer to string
- Convert data types of LotArea, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal from integer to float*
- Convert data types of BsmtFullBath, BsmtHalfBath from float to integer

***Code***

In [15]:
# Convert data type of MSSubClass to string
df_clean['MSSubClass'] = df_clean['MSSubClass'].astype(str)

# Convert data types of LotArea, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, WoodDeckSF, 
# OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal to float

columns = ['LotArea', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'WoodDeckSF', 'OpenPorchSF', 
           'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

for col in columns:
    df_clean[col] = df_clean[col].astype(float)
    
# Convert data types of BsmtFullBath, BsmtHalfBath to integer
columns = ['BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt']

for col in columns:
    df_clean[col] = df_clean[col].astype('Int64')

***Test***

In [16]:
df_clean.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856.0,854.0,0.0,1710.0,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003,RFn,2,548,TA,TA,Y,0.0,61.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262.0,0.0,0.0,1262.0,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976,RFn,2,460,TA,TA,Y,298.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920.0,866.0,0.0,1786.0,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001,RFn,2,608,TA,TA,Y,0.0,42.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961.0,756.0,0.0,1717.0,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998,Unf,3,642,TA,TA,Y,0.0,35.0,272.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145.0,1053.0,0.0,2198.0,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000,RFn,3,836,TA,TA,Y,192.0,84.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,12,2008,WD,Normal,250000


In [17]:
df[['MSSubClass', 'LotArea', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 
    'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 
    'MiscVal', 'BsmtFullBath', 'BsmtHalfBath']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   MSSubClass     1460 non-null   int64
 1   LotArea        1460 non-null   int64
 2   1stFlrSF       1460 non-null   int64
 3   2ndFlrSF       1460 non-null   int64
 4   LowQualFinSF   1460 non-null   int64
 5   GrLivArea      1460 non-null   int64
 6   WoodDeckSF     1460 non-null   int64
 7   OpenPorchSF    1460 non-null   int64
 8   EnclosedPorch  1460 non-null   int64
 9   3SsnPorch      1460 non-null   int64
 10  ScreenPorch    1460 non-null   int64
 11  PoolArea       1460 non-null   int64
 12  MiscVal        1460 non-null   int64
 13  BsmtFullBath   1460 non-null   int64
 14  BsmtHalfBath   1460 non-null   int64
dtypes: int64(15)
memory usage: 171.2 KB


In [18]:
df_clean[['MSSubClass', 'LotArea', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 
          'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 
          'MiscVal', 'BsmtFullBath', 'BsmtHalfBath']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   object 
 1   LotArea        1460 non-null   float64
 2   1stFlrSF       1460 non-null   float64
 3   2ndFlrSF       1460 non-null   float64
 4   LowQualFinSF   1460 non-null   float64
 5   GrLivArea      1460 non-null   float64
 6   WoodDeckSF     1460 non-null   float64
 7   OpenPorchSF    1460 non-null   float64
 8   EnclosedPorch  1460 non-null   float64
 9   3SsnPorch      1460 non-null   float64
 10  ScreenPorch    1460 non-null   float64
 11  PoolArea       1460 non-null   float64
 12  MiscVal        1460 non-null   float64
 13  BsmtFullBath   1460 non-null   Int64  
 14  BsmtHalfBath   1460 non-null   Int64  
dtypes: Int64(2), float64(12), object(1)
memory usage: 174.1+ KB


#### Step 2: Replace nulls in 23 variables

***Define***

Nulls in these 23 variables do not represent missing data, but they means that these variables are not possessed by the house. Out of 23 variables, 9 are numeric variables and 14 are categorical variables. Some of the variables are closely related to one anothers. For example, all the information about the garage of a house can be found from GarageYrBlt, GarageCars, GarageArea, GarageType, GarageFinish, GarageQual and GarageCond variables. If a house does not have a garage, then nulls in all the categorical variables that are related to garaga will be replaced with NoGarage. Whereas, nulls in all the numerical variables will be replaced with 0.

We can split all the 23 variables into 3 categories, which are:

- Basement (BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath, BsmtHalfBath, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2)
- Garage (GarageYrBlt, GarageCars, GarageArea, GarageType, GarageFinish, GarageQual, GarageCond)
- Others (Alley, FireplaceQu, PoolQC, Fence, MiscFeature)

The 3 categories will be cleaned separately with the sequence shown in the list above.

***Code***

In [19]:
# Clean basement category

# Define the variables that are related to basement
var_numerical = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']
var_categorical = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']

# Get boolean index of rows for house without basement
bool_index = df_clean[var_categorical].isnull().all(axis=1)

# Replace numeric variables with 0
df_clean.loc[bool_index, var_numerical] = 0

# Replace categorical variables with NoBsmt
df_clean.loc[bool_index, var_categorical] = 'NoBsmt'

In [20]:
# Clean garage category

# Define the variables that are related to garage
var_numerical = ['GarageYrBlt', 'GarageCars', 'GarageArea']
var_categorical = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']

# Get boolean index of rows for house without garage
bool_index = df_clean[var_categorical].isnull().all(axis=1)

# Replace numeric variables with 0
df_clean.loc[bool_index, var_numerical] = 0

# Replace categorical variables with NoBsmt
df_clean.loc[bool_index, var_categorical] = 'NoGarage'

In [21]:
# Clean other category

# Define the variables in other category
var_categorical = ['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

for var in var_categorical:
    # Get boolean index of rows with nulls
    bool_index = df_clean[var].isnull()
    
    # Replace categorical variables with No{Variable Name}
    df_clean.loc[bool_index, var] = 'No{}'.format(var)

***Test***

In [22]:
df_clean[['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtQual', 'BsmtCond', 
          'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'GarageType', 
          'GarageFinish', 'GarageQual', 'GarageCond', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   BsmtFinSF1    1460 non-null   int64 
 1   BsmtFinSF2    1460 non-null   int64 
 2   BsmtUnfSF     1460 non-null   int64 
 3   TotalBsmtSF   1460 non-null   int64 
 4   BsmtFullBath  1460 non-null   Int64 
 5   BsmtHalfBath  1460 non-null   Int64 
 6   BsmtQual      1460 non-null   object
 7   BsmtCond      1460 non-null   object
 8   BsmtExposure  1459 non-null   object
 9   BsmtFinType1  1460 non-null   object
 10  BsmtFinType2  1459 non-null   object
 11  GarageYrBlt   1460 non-null   Int64 
 12  GarageCars    1460 non-null   int64 
 13  GarageArea    1460 non-null   int64 
 14  GarageType    1460 non-null   object
 15  GarageFinish  1460 non-null   object
 16  GarageQual    1460 non-null   object
 17  GarageCond    1460 non-null   object
 18  Alley         1460 non-null   object
 19  Firepl

Based on the information shown in the previous cell, it can be seen that some variables that are related to basement still have nulls/missing data. Let's study the rows with nulls further.

In [23]:
# Get the rows that has nulls or missing data for basement-related variables
df_clean[df_clean[['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2']].isnull().any(axis=1)]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
332,333,20,RL,85.0,10655.0,Pave,NoAlley,IR1,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,1Story,8,5,2003,2004,Gable,CompShg,VinylSd,VinylSd,BrkFace,296.0,Gd,TA,PConc,Gd,TA,No,GLQ,1124,NaN,479,1603,3206,GasA,Ex,Y,SBrkr,1629.0,0.0,0.0,1629.0,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2003,RFn,3,880,TA,TA,Y,0.0,0.0,0.0,0.0,0.0,0.0,NoPoolQC,NoFence,NoMiscFeature,0.0,10,2009,WD,Normal,284000
948,949,60,RL,65.0,14006.0,Pave,NoAlley,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2002,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,144.0,Gd,TA,PConc,Gd,TA,NaN,Unf,0,Unf,0,936,936,GasA,Ex,Y,SBrkr,936.0,840.0,0.0,1776.0,0,0,2,1,3,1,Gd,7,Typ,1,TA,Attchd,2002,RFn,2,474,TA,TA,Y,144.0,96.0,0.0,0.0,0.0,0.0,NoPoolQC,NoFence,NoMiscFeature,0.0,2,2006,WD,Normal,192500


From the rows of data shown above, it can be observed that all the nulls in these variables actually represent missing data, instead of indicating that these variables are not possessed by the house. Missing data will be handled during Exploratory Data Analysis. Therefore, the objective of this particular cleaning process is achieved.

#### Step 3: Match the values of 'Exterior1st' and 'Exterior2nd'

***Define***

Change 'Wd Shng' in 'Exterior2nd' to 'WdShing' ('Exterior1st') and 'Brk Cmn' in 'Exterior2nd' to 'BrkComm' ('Exterior1st')

***Code***

In [24]:
# Replace 'Wd Shng' in 'Exterior2nd' with 'WdShing'
df_clean['Exterior2nd'] = df_clean['Exterior2nd'].replace('Wd Shng', 'WdShing') 

# Replace 'Brk Cmn' in 'Exterior2nd' with 'BrkComm'
df_clean['Exterior2nd'] = df_clean['Exterior2nd'].replace('Brk Cmn', 'BrkComm') 

***Test***

In [25]:
df_clean['Exterior2nd'].unique()

array(['VinylSd', 'MetalSd', 'WdShing', 'HdBoard', 'Plywood', 'Wd Sdng',
       'CmentBd', 'BrkFace', 'Stucco', 'AsbShng', 'BrkComm', 'ImStucc',
       'AsphShn', 'Stone', 'Other', 'CBlock'], dtype=object)

#### Step 4: Fix values of year-related variables

***Define***

- For rows with ID of 523, YearRemodAdd doesn't occur between YearBuilt and YrSold. YearRemodAdd should be replaced with YearBuilt or YrSold, depending on which is closer to YearRemodAdd

- For rows with IDs of 29, 93, 324, 600, 736, 1103, 1376, 1414 and 1418, GarageYrBlt doesn't occur between YearBuilt and YrSold. GarageYrBlt should be replaced with YearBuilt or YrSold, depending on which is closer to GarageYrBlt

***Code***

In [26]:
# Code for the first item in the list above

# Define the IDs to work on
ids_for_remod = 523

year_built = df.loc[ids_for_remod, 'YearBuilt']
year_sold = df.loc[ids_for_remod, 'YrSold']
year_remod_add = df.loc[ids_for_remod, 'YearRemodAdd']
    
# Compute absolute difference between YearBuilt and YearRemodAdd
diff_built_remod = abs(year_built - year_remod_add)
    
# Compute absolute difference between YearSold and YearRemodAdd
diff_sold_remod = abs(year_sold - year_remod_add)
    
# YearRemodAdd should is replaced with YearBuilt or YrSold, depending on which is closer to YearRemodAdd
df_clean.loc[ids_for_remod, 'YearRemodAdd'] = year_built if diff_built_remod <= diff_sold_remod else year_sold

In [27]:
# Code for the second item in the list above

# Define the IDs to work on
ids_for_garage = [29, 93, 324, 600, 736, 1103, 1376, 1414, 1418]

for row_id in ids_for_garage:
    year_built = df.loc[row_id, 'YearBuilt']
    year_sold = df.loc[row_id, 'YrSold']
    year_garage_built = df.loc[row_id, 'GarageYrBlt']
    
    # Compute absolute difference between YearBuilt and YearRemodAdd
    diff_built_garage = abs(year_built - year_garage_built)
    
    # Compute absolute difference between YearSold and YearRemodAdd
    diff_sold_garage = abs(year_sold - year_garage_built)
    
    
    df_clean.loc[row_id, 'GarageYrBlt'] = year_built if diff_built_garage <= diff_sold_garage else year_sold

***Test***

In [28]:
# Test for the first item in the list above
df_clean.loc[[ids_for_remod] , ['YearBuilt', 'YearRemodAdd', 'YrSold']]

,YearBuilt,YearRemodAdd,YrSold
523,2007,2007,2007


In [29]:
# Test for the second item in the list above
df_clean.loc[ids_for_garage , ['YearBuilt', 'GarageYrBlt', 'YrSold']]

,YearBuilt,GarageYrBlt,YrSold
29,1927,1927,2008
93,1910,1910,2007
324,1967,1967,2010
600,2005,2005,2006
736,1950,1950,2006
1103,1959,1959,2006
1376,1930,1930,2008
1414,1923,1923,2008
1418,1963,1963,2008


#### Step 5: Drop variables with too many missing data

***Define***

- Drop Alley, FireplaceQu, PoolQC, Fence and MiscFeature from the data frame due to too many missing data

***Code***

In [30]:
# Define columns to be dropped
cols = ['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

df_clean = df_clean.drop(cols, axis=1)

***Test***

In [31]:
# Verify that columns are dropped from the data frame
print(set(cols) & set(df_clean.columns))

set()


## Data Saving <a class="anchor" id="saving"></a>

After data wrangling is completed, the cleaned data frame is saved to a CSV file

In [32]:
# Save cleaned dataframe to a CSV file
df_clean.to_csv('train_cleaned.csv', index=False)